<a href="https://colab.research.google.com/github/JeroAgullo/fundamental_valuation/blob/main/fundamental_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Fundamental Valuation using Python

## 1.1. Introduction

**Project Approach**  
This project provides a fundamental valuation of NVIDIA Corporation (NVDA) using Python tools and financial data sourced from the Yahoo Finance API via the **yfinance** library.

The notebook parses and standardizes key financial statement elements into a format that facilitates **ratio analysis and valuation modeling**.

**Fundamental Analysis**  
Fundamental analysis is a method used in accounting and finance to estimate the **intrinsic value of a security** in order to help with investment decisions.

Unlike technical analysis, which focuses on price trends, fundamental analysis seeks to determine the "true" value of a security to identify whether it is overvalued, undervalued, or fairly valued **compared with its current market price**.

## 1.2. First steps

We will start by importing the needed libraries. Then, we are creating an object for the chosen company using yfinance's function '.Ticker', allowing us to get all its public financial information.

We are using NVIDIA's ticker (the market "ID") for this project, but any other ticker can be applied. Here is a list of companies in the S&P500 Index, and their ticker:

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

In [ ]:
# Libraries import
import yfinance as yf
import pandas as pd

In [ ]:
# NVIDIA object creation
ticker = yf.Ticker("NVDA")

In [ ]:
# Connection check
headers = ticker.balance_sheet.index.tolist() # We will print the headers of the Income Statement report
for h in headers:
  print(h)

Treasury Shares Number
Ordinary Shares Number
Share Issued
Net Debt
Total Debt
Tangible Book Value
Invested Capital
Working Capital
Net Tangible Assets
Capital Lease Obligations
Common Stock Equity
Total Capitalization
Total Equity Gross Minority Interest
Stockholders Equity
Gains Losses Not Affecting Retained Earnings
Other Equity Adjustments
Treasury Stock
Retained Earnings
Additional Paid In Capital
Capital Stock
Common Stock
Preferred Stock
Total Liabilities Net Minority Interest
Total Non Current Liabilities Net Minority Interest
Other Non Current Liabilities
Employee Benefits
Tradeand Other Payables Non Current
Non Current Deferred Liabilities
Non Current Deferred Revenue
Non Current Deferred Taxes Liabilities
Long Term Debt And Capital Lease Obligation
Long Term Capital Lease Obligation
Long Term Debt
Current Liabilities
Other Current Liabilities
Current Deferred Liabilities
Current Deferred Revenue
Current Debt And Capital Lease Obligation
Current Capital Lease Obligation
Curre

## 1.3. Data extract

We are now getting all the necessary data to proceed with the fundamental analysis. Not all the accessible information is needed, so we are selecting specific one from each Financial State (Financials):

- Income Statement
- Balance Sheet
- Cashflow

The market information (such as price or number of shares) will be also consulted for the more advanced financial valuations.

In [ ]:
# Auxiliar function to get only existing values and to limit error cases
def safe_extract(df, row_name, column_name):
    if row_name in df.index:
        return df.loc[row_name, column_name]
    else:
        return None

In [ ]:
# Data extract - Income Statement
df = ticker.ttm_financials
column_name = df.columns[0] # First column from the TTM (Trailing Twelve Months) report

total_revenue = safe_extract(df, "Total Revenue", column_name)
gross_profit = safe_extract(df, "Gross Profit", column_name)
ebitda = safe_extract(df, "EBITDA", column_name)
ebit = safe_extract(df, "EBIT", column_name)
net_income = safe_extract(df, "Net Income", column_name)

In [ ]:
# Data extract - Balance Sheet
df= ticker.balancesheet
column_name = df.columns[0] # First column, to get only the latest accounting period report

total_assets = safe_extract(df, "Total Assets", column_name)
current_assets = safe_extract(df, "Current Assets", column_name)
inventory = safe_extract(df, "Inventory", column_name)
total_liabilities = safe_extract(df, "Total Liabilities Net Minority Interest", column_name)
current_liabilities = safe_extract(df, "Current Liabilities", column_name)
total_equity = safe_extract(df, "Total Equity Gross Minority Interest", column_name)
cash = safe_extract(df, "Cash And Cash Equivalents", column_name)
ordinary_shares = safe_extract(df, "Ordinary Shares Number", column_name)

In [ ]:
# Data extract - Cashflow
df = ticker.ttm_cash_flow
column_name = df.columns[0] # First column from the TTM (Trailing Twelve Months) report

cashflow = safe_extract(df, "Operating Cash Flow", column_name)
free_cashflow = safe_extract(df, "Free Cash Flow", column_name)

In [ ]:
# Data extract - Market information
df = ticker.info

current_price = df["currentPrice"]
current_shares = df["sharesOutstanding"]
current_market_cap = df["marketCap"]

In [ ]:
# Results print
print("Income Statement results")
print(f"Total Revenue: {total_revenue:,.0f}$")
print(f"Total Profit: {gross_profit:,.0f}")
print(f"EBITDA: {ebitda:,.0f}$")
print(f"EBIT: {ebit:,.0f}$")
print(f"Net Income: {net_income:,.0f}$")

print("\nBalance Sheet results")
print(f"Total Assets: {total_assets:,.0f}$")
print(f"Total Liabilities: {total_liabilities:,.0f}$")
print(f"Total Equity: {total_equity:,.0f}$")
print(f"Cash: {cash:,.0f}$")


print("\nCashflow results")
print(f"Cash Flow: {cashflow:,.0f}$")
print(f"Free Cash Flow: {free_cashflow:,.0f}$")

print("\nMarket information")
print(f"Current Price: {current_price:,.2f}$")
print(f"Current Shares: {current_shares:,.0f}")
print(f"Current Market Cap: {current_market_cap:,.0f}$")

Income Statement results
Total Revenue: 148,515,000,000$
Total Profit: 104,120,000,000
EBITDA: 90,968,000,000$
EBIT: 88,903,000,000$
Net Income: 76,774,000,000$

Balance Sheet results
Total Assets: 111,601,000,000$
Total Liabilities: 32,274,000,000$
Total Equity: 79,327,000,000$
Cash: 8,589,000,000$

Cashflow results
Cash Flow: 76,158,000,000$
Free Cash Flow: 72,064,000,000$

Market information
Current Price: 173.72$
Current Shares: 24,387,600,384
Current Market Cap: 4,236,614,041,600$


# 2. Financial Ratios

## 2.1. Introduction

Financial ratios are quantitative metrics derived from Financial Statements' data, allowing us to simplify the interpretation of such raw information.

These ratios will gives us a clear data-driven picture of the studied security and its past and current health, being a key part of fundamental valuation. They can be structured as followed:

- Leverage and Liquidity Ratios
- Profitability Ratios
- Valuation Ratios (Multiples)

## 2.2. Leverage and Liquidity Ratios

Indicators that let us know the company's the strength of its capital structure and the capacity to attend to its long-term (leverage) and its short-term obligations (liquidity).

In [ ]:
# Leverage Ratios
debt_equity = total_liabilities / total_equity
print(f"Debt to Equity (D/E) Ratio: {debt_equity:.2%}")

debt_asset = total_liabilities / total_assets
print(f"Debt to Asset (D/A) Ratio: {debt_asset:.2%}")

Debt to Equity (D/E) Ratio: 40.68%
Debt to Asset (D/A) Ratio: 28.92%


In [ ]:
# Liquidity Ratios
current = current_assets / current_liabilities
print(f"Current Ratio: {current:.2%}")

quick = (current_assets - inventory) / current_liabilities
print(f"Quick (Acid) Ratio: {quick:.2%}")

cash = cash / current_liabilities
print(f"Cash Ratio: {cash:.2%}")

Current Ratio: 443.99%
Quick (Acid) Ratio: 388.13%
Cash Ratio: 47.59%


## 2.3. Profitability Ratios

Indicators that let us know, not only if the company can attend to its obligations, but also if it gives profit to its owners.

In [ ]:
# Return Ratios
roi = net_income / total_revenue
print(f"Return on Investment (ROI): {roi:.2%}")

roe = net_income / total_equity
print(f"Return on Equity (ROE): {roe:.2%}")

roa = net_income / total_assets
print(f"Return on Assets (ROA): {roa:.2%}")

Return on Investment (ROI): 51.69%
Return on Equity (ROE): 96.78%
Return on Assets (ROA): 68.79%


In [ ]:
# Efficency Ratios
gross_margin = gross_profit / total_revenue
print(f"Gross Margin: {gross_margin:.2%}")

ebitda_margin = ebitda / total_revenue
print(f"EBITDA Margin: {ebitda_margin:.2%}")

fcf_revenue = free_cashflow / total_revenue
print(f"Free Cash Flow Margin: {fcf_revenue:.2%}")

Gross Margin: 70.11%
EBITDA Margin: 61.25%
Free Cash Flow Margin: 48.52%


In [ ]:
# Share Ratios
bvps = total_equity / ordinary_shares # We use the share number reported in the same period as the Equity (Accounting Period)
print(f"Book Value per Share (BVPS): {bvps:.2f}$")

eps = net_income / current_shares # We use the share number reported in the same period as the Net Income (TTM)
print(f"Earnings per Share (EPS): {eps:.2f}$")

cfps = cashflow / current_shares # We use the share number reported in the same period as the Cash Flow (TTM)
print(f"Cash Flow per Share (CFPS): {cfps:.2f}$")

Book Value per Share (BVPS): 3.24$
Earnings per Share (EPS): 3.15$
Cash Flow per Share (CFPS): 3.12$


## 2.4. Valuation Ratios (Multiples)

Indicators most commonly used in financial investments. They allow us to compare companies from the same sector and competitors in the market.

In [ ]:
# Price to Book Value (PBV)
p_bv = current_price / bvps
print(f"Price to Book Value (P/BV): x{p_bv:.2f}")

Price to Book Value (P/BV): x53.60


In [ ]:
# Price to Earnings Ratio (PER)
per = current_price / eps
print(f"Price to Earnings Ratio (PER or P/E): x{per:.2f}")

Price to Earnings Ratio (PER or P/E): x55.18


In [ ]:
# Enterprise Value to EBITDA (EV/EBITDA)
enterprise_value = current_market_cap + total_liabilities - cash # Enterprise Value (EV) must be calculated first
ev_ebitda = enterprise_value / ebitda
print(f"Enterprise Value to EBITDA (EV/EBITDA): x{ev_ebitda:.2f}")

Enterprise Value to EBITDA (EV/EBITDA): x46.93


In [ ]:
# Price to Cash Flow per share (PCF)
pcf = current_price / cfps
print(f"Price to Cash Flow (PCF): x{pcf:.2f}")

Price to Cash Flow (PCF): x55.63
